In [8]:
from lxml import etree
import re
import pandas as pd
import multiprocessing
import itertools

def process_element(elem):    
    if elem[1].text == '0':
        text = elem[-1][-2].text
        if text is not None:
            text=text.lower()
            if "#REDIRECT [[" in text:
                return None
            title = elem[0].text
            length = len(text)
            is_dis = ("{{disambig}}" in text) or ('{{disambiguation}}' in text) or ('{{dab}}' in text) or ('{{disamb}}' in text)
            is_stub = ("{{Stub}}" in text) or ("-stub}}" in text)
            return (title, length, is_dis, is_stub )
    return None


def scan(dump):
    print "starting scan of dump %s" % dump
    i = 0
    context = etree.iterparse( dump , tag = '{http://www.mediawiki.org/xml/export-0.10/}page')
    l = []
    for event, elem in context:
        i+=1
        result_tuple = process_element(elem)
        if result_tuple is not None:
            l.append(result_tuple)
        elem.clear()
        for ancestor in elem.xpath('ancestor-or-self::*'):
            while ancestor.getprevious() is not None:
                del ancestor.getparent()[0]
    del context
    print "finishing scan of dump %s" % dump
    
    return l


def scan_dump_parrallel(wikilang, n):
    exp_dir = 'en-%s' % wikilang
    args = ['/home/ellery/%s/dump%d.xml' % (exp_dir, i) for i in range(1, n+1)]
    p = multiprocessing.Pool(8)
    results = p.map(scan, args)
    l = list(itertools.chain(*results))
    df = pd.DataFrame(l)
    df.columns = ['title', 'length', 'is_dis', 'is_stub']
    return df

df = scan_dump_parrallel('en', 27)
df.to_csv('/home/ellery/en-en/dump_features.tsv',  sep = '\t', encoding = 'utf8')
print df.shape

(11571224, 4)
starting scan of dump /home/ellery/en-en/dump1.xml
starting scan of dump /home/ellery/en-en/dump4.xml
starting scan of dump /home/ellery/en-en/dump7.xml
starting scan of dump /home/ellery/en-en/dump5.xml
starting scan of dump /home/ellery/en-en/dump6.xml
starting scan of dump /home/ellery/en-en/dump2.xml
starting scan of dump /home/ellery/en-en/dump8.xml
starting scan of dump /home/ellery/en-en/dump3.xml
finishing scan of dump /home/ellery/en-en/dump1.xmlfinishing scan of dump /home/ellery/en-en/dump4.xmlfinishing scan of dump /home/ellery/en-en/dump7.xmlfinishing scan of dump /home/ellery/en-en/dump5.xmlfinishing scan of dump /home/ellery/en-en/dump6.xmlfinishing scan of dump /home/ellery/en-en/dump2.xmlfinishing scan of dump /home/ellery/en-en/dump8.xmlfinishing scan of dump /home/ellery/en-en/dump3.xml







starting scan of dump /home/ellery/en-en/dump9.xml
starting scan of dump /home/ellery/en-en/dump12.xmlstarting scan of dump /home/ellery/en-en/dump15.xmlstarting 

In [9]:
df = pd.read_csv('/home/ellery/en-en/dump_features.tsv',  sep = '\t', encoding = 'utf8')

In [10]:
df.head()

,Unnamed: 0,title,length,is_dis,is_stub
0,0,AccessibleComputing,69,False,False
1,1,Anarchism,176778,False,False
2,2,AfghanistanHistory,57,False,False
3,3,AfghanistanGeography,59,False,False
4,4,AfghanistanPeople,62,False,False


In [11]:
df.shape

(11571224, 5)

In [12]:
df['is_dis'].sum()

144555

In [13]:
df['is_stub'].sum()

1895560

In [14]:
df[df['is_stub']][:10]

,Unnamed: 0,title,length,is_dis,is_stub
75,75,Actrius,1747,False,True
181,181,List of anthropologists,6179,False,True
268,268,Analog Brothers,2611,False,True
334,334,American shot,1719,False,True
382,382,Northern cavefish,2408,False,True
416,416,Geography of American Samoa,2964,False,True
420,420,Transportation in American Samoa,1372,False,True
428,428,America's National Game,810,False,True
576,576,Ancylopoda,1693,False,True
749,749,Alexander Aetolus,4052,False,True
